#  불러오기 

In [2]:
import pandas as pd

In [25]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [67]:
test.head()


,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0


In [29]:
## 열평균으로 결측치 대체
# train.fillna({'hour_bef_temperature':int(train['hour_bef_temperature'].mean())}, inplace=True)
# train.fillna({'hour_bef_precipitation':int(train['hour_bef_precipitation'].mean())}, inplace=True)
# train.fillna({'hour_bef_windspeed':int(train['hour_bef_windspeed'].mean())}, inplace=True)
# train.fillna({'hour_bef_humidity':int(train['hour_bef_humidity'].mean())}, inplace=True)
# train.fillna({'hour_bef_visibility':int(train['hour_bef_visibility'].mean())}, inplace=True)
# train.fillna({'hour_bef_ozone':int(train['hour_bef_ozone'].mean())}, inplace=True)
# train.fillna({'hour_bef_pm10':int(train['hour_bef_pm10'].mean())}, inplace=True)
# train.fillna({'hour_bef_pm2.5':int(train['hour_bef_pm2.5'].mean())}, inplace=True)

# 보간법으로 결측치 대체
train.interpolate(inplace=True)
test.interpolate(inplace=True)

In [35]:
train.shape

(1459, 11)

# 모델구현

In [36]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

In [63]:
# X_train = train.drop(['count'], axis=1)
Y_train = train['count']

model = RandomForestRegressor(criterion = 'mse')
model.fit(X_train, Y_train)


C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse')

In [41]:
model.feature_importances_

array([0.02663803, 0.59548078, 0.17836277, 0.01928792, 0.02593601,
       0.03656834, 0.03242656, 0.03372985, 0.03109853, 0.02047121])

In [68]:
# X_train 에서 drop 할 피쳐의 경우에 수 대로 3개의 X_train 을 생성하세요.

X_train_1 = train.drop(['count','id'], axis=1)
X_train_2 = train.drop(['count', 'id', 'hour_bef_windspeed'], axis=1)
X_train_3 = train.drop(['count', 'id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

# 각 train 에 따라 동일하게 피쳐를 drop 한 test 셋들을 생성하세요.

test_1 = test.drop(['id'], axis=1)
test_2 = test.drop(['id', 'hour_bef_windspeed'], axis=1)
test_3 = test.drop(['id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

In [69]:

model_input_var1 = RandomForestRegressor(criterion = 'mse')
model_input_var1.fit(X_train_1, Y_train)

model_input_var2 = RandomForestRegressor(criterion = 'mse')
model_input_var2.fit(X_train_2, Y_train)

model_input_var3 = RandomForestRegressor(criterion = 'mse')
model_input_var3.fit(X_train_3, Y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse')

In [70]:
y_pred_1 = model_input_var1.predict(test_1)
y_pred_2 = model_input_var2.predict(test_2)
y_pred_3 = model_input_var3.predict(test_3)

In [73]:
model_input_var3.feature_importances_

array([0.60240161, 0.19557898, 0.0190468 , 0.04600996, 0.04167676,
       0.04812943, 0.04715646])

# 저장

In [74]:

submission_1 = pd.read_csv('submission.csv')
submission_2 = pd.read_csv('submission.csv')
submission_3 = pd.read_csv('submission.csv')

submission_1['count'] = y_pred_1
submission_2['count'] = y_pred_2
submission_3['count'] = y_pred_3

submission_1.to_csv('sub_1.csv',index=False)
submission_2.to_csv('sub_2.csv',index=False)
submission_3.to_csv('sub_3.csv',index=False)

# GridSearch

In [76]:
from sklearn.model_selection import GridSearchCV


model = RandomForestRegressor(criterion = 'mse', random_state=2020)


params = {'n_estimators': [200, 300, 500],
          'max_features': [5, 6, 8],
          'min_samples_leaf': [1, 3, 5]}


greedy_CV = GridSearchCV(model, param_grid=params, cv = 3, n_jobs = -1)
greedy_CV.fit(X_train_3, Y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
27 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\

GridSearchCV(cv=3,
             estimator=RandomForestRegressor(criterion='mse',
                                             random_state=2020),
             n_jobs=-1,
             param_grid={'max_features': [5, 6, 8],
                         'min_samples_leaf': [1, 3, 5],
                         'n_estimators': [200, 300, 500]})

In [79]:
pred = greedy_CV.predict(test_3)

In [80]:
pred

array([105.54333333, 214.12      ,  84.47      ,  40.77      ,
        59.20333333, 123.42333333, 194.43333333, 317.70666667,
        42.04      , 111.56333333, 295.17666667, 250.64      ,
        88.01666667,  41.04      , 208.23333333, 142.61      ,
        26.64      , 171.33333333, 348.24333333, 153.84      ,
       232.16      ,  85.77333333,  25.5       , 140.47333333,
       142.61      , 114.72      ,  29.69666667, 116.35666667,
       112.89666667, 148.77      ,  75.32      ,  34.78333333,
        62.47      , 130.01      , 267.44333333,  40.01666667,
       129.84333333, 103.61666667, 244.90333333,  91.13      ,
        54.85      , 122.29      , 152.97      ,  91.05333333,
       333.61333333, 216.60333333,  94.61333333,  59.33333333,
        18.46666667,  85.43      , 226.15666667,  91.92666667,
       178.04      ,  82.18333333, 187.81666667, 131.50333333,
        51.48      , 171.97666667,  22.02      ,  20.24333333,
        88.26      ,  88.27      , 255.10333333, 307.49

In [81]:
submission = pd.read_csv('submission.csv')

In [82]:
import numpy as np

submission['count'] = np.round(pred, 2)

In [83]:
submission.head()

,id,count
0,0,105.54
1,1,214.12
2,2,84.47
3,4,40.77
4,5,59.20


In [84]:
submission.to_csv('sub.csv',index=False)